# Neural network

- $Z^{[1]} = w^{[1]}X + b^{[1]}$
- $A^{[1]} = \sigma(Z^{[1]})$
- $Z^{[2]} = w^{[2]}A^{[1]} + b^{[2]}$
- $A^{[2]} = \sigma(Z^{[2]})$
    - Where 
        - $X$: $(n_{x}, m)$ matrix
        - $Z$: $($number of hidden units, $m)$ matrix
        - $A$: $($number of hidden units, $m)$ matrix
        
## Activation function

- Binary classification? use sigmoid
- All other cases? use RELU (rectified linear unit)

Why use non-linear activation function?
- If use linear activation funciotn, having layers becomes meaningless because combinations of linear function reduce down to a single linear function

## Gradient descent for neural networks

Parameters
- $n_{x} = n^{[0]}$: number of features
- $n^{[1]}$: number of hidden units
- $n^{[2]} = 1$: number of output units
- $w^{[1]}$: $(n^{[1]}, n^{[0]})$ matrix
- $b^{[1]}$: $(n^{[1]}, 1)$ matrix
- $w^{[2]}$: $(n^{[2]}, n^{[1]})$ matrix
- $b^{[2]}$: $(n^{[2]}, 1)$ matrix

Cost function
- $J(w^{[1]}, b^{[1]}, w^{[2]}, b^{[2]}) = \dfrac{1}{m}\displaystyle\sum_{i=1}^{n}L(\hat{y}, y)$ where $L(\hat{y}, y) = a^{[2]}$

Gradient descent
- Repeat
    - Compute prediction $\hat{y}^{(i)}$ for $i = 1 \dots m$
    - $dw^{[1]} = \dfrac{\partial J}{\partial w^{[1]}}$, $db^{[1]} = \dfrac{\partial J}{\partial b^{[1]}}$, $dw^{[2]} = \dfrac{\partial J}{\partial w^{[2]}}$, $db^{[2]} = \dfrac{\partial J}{\partial b^{[2]}}$
    - $w^{[1]} = w^{[1]} - \alpha dw^{[1]}$, $b^{[1]} = b^{[1]} - \alpha db^{[1]}$, $w^{[2]} = w^{[2]} - \alpha dw^{[2]}$, $b^{[2]} = b^{[2]} - \alpha db^{[2]}$
    
Backward propagation
- $dZ^{[2]} = A^{[2]} - Y$
- $dw^{[2]} = \dfrac{1}{m}dZ^{[2]}A^{[1]^{T}}$
- $db^{[2]} = \dfrac{1}{m}$np.sum$(dZ^{[2]}$, axis=1, keepdims=True$)$
- $dZ^{[1]} = w^{[2]^{T}}dZ^{[2]} * g^{[1]^{'}}(Z^{[1]})$
- $dw^{[1]} = \dfrac{1}{m}dZ^{[1]}X^{T}$
- $db^{[1]} = \dfrac{1}{m}$np.sum$(dZ^{[1]}$, axis=1, keepdims=True$)$


## Example

<img src="img/classification_kiank.png" style="width:600px;height:300px;">

### Packages

- [sklearn](http://scikit-learn.org/stable/) provides simple and efficient tools for data mining and data analysis. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model

from dl_utils import *

%matplotlib inline

### Data

In [ ]:
X, Y = load_planar_dataset()

In [ ]:
def layer_sizes(X, Y):
    """
    Arguments:
    X -- Input dataset of shape. (input size, number of examples)
    Y -- Labels of shape (output size, number of examples)
    
    Returns:
    n_x -- The size of the input layer.
    n_h -- The size of the hidden layer.
    n_y -- The size of the output layer.
    """

    n_x = X.shape[0] # size of input layer
    n_h = 4
    n_y = Y.shape[0] # size of output layer

    return (n_x, n_h, n_y)

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- Size of the input layer.
    n_h -- Size of the hidden layer.
    n_y -- Size of the output layer.
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape. (n_h, n_x)
                    b1 -- bias vector of shape. (n_h, 1)
                    W2 -- weight matrix of shape. (n_y, n_h)
                    b2 -- bias vector of shape. (n_y, 1)
    """
    
    np.random.seed(2)  # we set up a seed so that we get expected output.
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- Input data of size. (n_x, m)
    parameters -- Python dictionary containing your parameters. (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation.
    cache -- A dictionary containing "Z1", "A1", "Z2" and "A2".
    """
    
    # Retrieve each parameter from the dictionary "parameters".
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Implement Forward Propagation to calculate A2. (probabilities)
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [ ]:
def compute_cost(A2, Y, parameters):
    """
    Computes the cross-entropy cost.
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape. (1, number of examples)
    Y -- "true" labels vector of shape. (1, number of examples)
    parameters -- Python dictionary containing your parameters W1, b1, W2 and b2.
    
    Returns:
    cost -- Cross-entropy cost given equation.
    """
    
    m = Y.shape[1]  # Number of example.

    # Compute the cross-entropy cost.
    logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1-A2), (1-Y))
    cost = - np.sum(logprobs) / m
    
    # Makes sure cost is the dimension we expect. 
    # E.g. turns [[17]] into 17 .
    cost = float(np.squeeze(cost))  
    assert(isinstance(cost, float))
    
    return cost

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- Python dictionary containing our parameters. 
    cache -- A dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- Input data of shape. (2, number of examples)
    Y -- "true" labels vector of shape. (1, number of examples)
    
    Returns:
    grads -- Python dictionary containing your gradients with respect to different parameters.
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    W1 = parameters["W1"]
    W2 = parameters["W2"]
        
    # Retrieve also A1 and A2 from dictionary "cache".
    A1 = cache["A1"]
    A2 = cache["A2"]

    # Backward propagation: calculate dW1, db1, dW2, db2. 
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m 
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [ ]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    Updates parameters using the gradient descent update rule.
    
    Arguments:
    parameters -- Python dictionary containing your parameters. 
    grads -- Python dictionary containing your gradients. 
    
    Returns:
    parameters -- Python dictionary containing your updated parameters. 
    """
    
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    # Retrieve each gradient from the dictionary "grads"
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    # Update rule for each parameter
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    """
    Arguments:
    X -- Dataset of shape. (2, number of examples)
    Y -- Labels of shape. (1, number of examples)
    n_h -- Size of the hidden layer.
    num_iterations -- Number of iterations in gradient descent loop.
    print_cost -- If True, print the cost every 1000 iterations.
    
    Returns:
    parameters -- Parameters learnt by the model, which can be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters.
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    # Loop. (gradient descent)

    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, Y, parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads, learning_rate = 1.2)
        
        # Print the cost every 1000 iterations.
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [ ]:
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X.
    
    Arguments:
    parameters -- Python dictionary containing your parameters. 
    X -- Input data of size. (n_x, m)
    
    Returns
    predictions -- Vector of predictions of our model. (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 > 0.5)
    
    return predictions

In [ ]:
# Build a model with a n_h-dimensional hidden layer
parameters = nn_model(X, Y, n_h = 4, num_iterations = 10000, print_cost=True)

# Plot the decision boundary
plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y.reshape(Y.shape[1]))
plt.title("Decision Boundary for hidden layer size " + str(4))

# Print accuracy
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')

### Tuning hidden layer size

- Larger models (with more hidden units) are able to fit the training set better, until eventually the largest models overfit the data. 
- The best hidden layer size seems to be around `n_h = 5`. Indeed, a value around here seems to  fits the data well without also incurring noticeable overfitting.

In [ ]:
plt.figure(figsize=(16, 32))
hidden_layer_sizes = [1, 2, 3, 4, 5, 20, 50]
for i, n_h in enumerate(hidden_layer_sizes):
    plt.subplot(5, 2, i+1)
    plt.title('Hidden Layer of size %d' % n_h)
    parameters = nn_model(X, Y, n_h, num_iterations = 5000)
    plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y.reshape(Y.shape[1]))
    predictions = predict(parameters, X)
    accuracy = float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100)
    print ("Accuracy for {} hidden units: {} %".format(n_h, accuracy))